In [1]:
from deepeval.test_case import LLMTestCase, ConversationalTestCase
from deepeval.synthesizer import Synthesizer
from deepeval.dataset import Golden
from deepeval.synthesizer.config import StylingConfig
from repositories.faq import get_all as get_all_faqs
from models.faq import FAQModel
import json
from typing import List, Dict
import random
import openai
from openai import OpenAI

In [3]:
# Initialize OpenAI client
client = OpenAI(api_key="sk-BJpqmcWfn3O9bvr2eryf6Uyf1m7VpSQpL7cNxGGlZccRrfA1",base_url="https://open.keyai.shop/v1")
faqs = get_all_faqs()


In [8]:
faqs_questions = [faq.question for faq in faqs]
faq_questions_string = "\\n-".join([f"- {question}" for question in faqs_questions])
prompt_template = """
Bạn là một Chuyên gia Thiết kế Test Case AI, chuyên tạo ra các kịch bản hội thoại đa dạng và thực tế để đánh giá chatbot tiếng Việt.

NHIỆM VỤ:
Mục tiêu của bạn là tạo ra một bộ dữ liệu gồm chính xác 30 kịch bản `ConversationalTestCase` . Mỗi kịch bản sẽ được sử dụng để kiểm tra khả năng của chatbot trong việc xử lý các truy vấn khác nhau từ người dùng và cách sử dụng đại từ nhân xưng đúng đắn dựa trên user_gender hoặc pronoun trong user_input.

DANH SÁCH CÂU HỎI FAQ THAM KHẢO (Sử dụng để simulate các câu hỏi của người dùng):
{faq_questions_string}

YÊU CẦU CHI TIẾT CHO MỖI `ConversationalTestCase`:
Mỗi `ConversationalTestCase` phải là một đối tượng JSON và có các đặc điểm sau:
1.  Một trường `conversation_id` (chuỗi, ví dụ: "conv_01", "conv_02", ... , "conv_30").
2.  Một trường `turns` (danh sách). Danh sách này phải chứa từ 1 đến 4 đối tượng, mỗi đối tượng đại diện cho một lượt của người dùng và sẽ được dùng để tạo một `LLMTestCase` sau này.
3.  Đối với mỗi đối tượng "turn" trong danh sách `turns`:
    a.  `user_input` (chuỗi): Tạo một câu hỏi tự nhiên từ phía người dùng. Câu hỏi này NÊN được lấy cảm hứng từ hoặc dẫn dắt đến một trong các câu hỏi trong "DANH SÁCH CÂU HỎI FAQ THAM KHẢO" ở trên.
        -   **Quan trọng:** Câu hỏi của người dùng PHẢI sử dụng đa dạng các đại từ nhân xưng tiếng Việt để tự xưng, ví dụ: "anh", "chị", "cô", "chú", "bác", "em", "tui", "tôi", "mình".
        -   Cũng bao gồm các trường hợp người dùng KHÔNG sử dụng đại từ nhân xưng rõ ràng (ví dụ: "Làm thế nào để...", "Cho hỏi về...", "Sản phẩm này còn không?").
        -   Đảm bảo sự đa dạng và phân bổ hợp lý các loại đại từ này trong toàn bộ 30 kịch bản.
        -   Ví dụ: "Anh muốn hỏi cách tra cứu về hóa đơn đã mua hàng tại FPT Shop?", "Làm thế nào để kiểm tra được tình trạng máy đã gửi đi bảo hành tại FPT Shop?", "Em ơi, chị cần hỗ trợ về vấn đề tài khoản.", "Chú hỏi chút, cửa hàng mình có chính sách cho khách hàng thân thiết không?"
    b.  `user_gender_context` (chuỗi hoặc null): Chỉ định ngữ cảnh giới tính của người dùng cho lượt đó. Giá trị có thể là một trong các chuỗi: "male", "female", "unknown". Trong một số trường hợp, giá trị này có thể là `null` (hoặc trường này có thể không xuất hiện), cho biết không có thông tin giới tính cụ thể nào được cung cấp cho lượt đó. Phân bổ các tùy chọn này một cách thực tế và đa dạng.
    c.  `bot_actual_output_placeholder` (chuỗi): Trường này PHẢI LUÔN LUÔN là một chuỗi rỗng (`""`). Nó sẽ được điền vào sau bởi phản hồi thực tế của chatbot.

YÊU CẦU CHO TOÀN BỘ BỘ DỮ LIỆU (30 `ConversationalTestCase`):
1.  Tạo ra CHÍNH XÁC 30 kịch bản `ConversationalTestCase`.
2.  Đảm bảo sự đa dạng cao trong toàn bộ 30 kịch bản về các mặt sau:
    * Bắt buộc dựa trên DANH SÁCH CÂU HỎI FAQ THAM KHẢO gốc ở trên.
    * Số lượng "turns" trong mỗi hội thoại (từ 1 đến 4).
    * Đại từ nhân xưng được sử dụng trong `user_input`.
    * Giá trị `user_gender_context` được cung cấp cho mỗi lượt.
    * Phong cách hỏi của người dùng (trang trọng, thân mật, trực tiếp, v.v.).

ĐỊNH DẠNG ĐẦU RA:
Trả về TOÀN BỘ bộ dữ liệu dưới dạng MỘT MẢNG JSON DUY NHẤT (a single JSON array). Mỗi phần tử của mảng là một đối tượng JSON đại diện cho một `ConversationalTestCase`, tuân thủ cấu trúc đã mô tả ở trên (với các khóa `conversation_id`, `turns`; và trong mỗi turn là `user_input`, `user_gender_context`, `bot_actual_output_placeholder`).

VÍ DỤ VỀ CẤU TRÚC CHO MỘT `ConversationalTestCase` TRONG MẢNG JSON:
```json
{
  "conversation_id": "conv_01",
  "turns": [
    {
      "user_input": "Anh muốn hỏi cách tra cứu về hóa đơn đã mua hàng tại FPT Shop?",
      "user_gender_context": "male",
      "bot_actual_output_placeholder": ""
    },
    {
      "user_input": "Nếu hóa đơn đó mua hơn 1 năm rồi thì có tìm lại được không shop?",
      "user_gender_context": "male",
      "bot_actual_output_placeholder": ""
    }
  ]
}

"""

#final_prompt = prompt_template.format(faq_questions_string=faq_questions_string)
final_prompt = prompt_template.replace("{faq_questions_string}", faq_questions_string)


In [9]:
response = client.chat.completions.create(
        model="gpt-4o", 
        messages=[
            {"role": "system", "content": "Bạn là một AI hỗ trợ tạo dữ liệu test case cho deepeval đánh giá chatbot tiếng việt."},
            {"role": "user", "content": final_prompt}
        ],
         # response_format={"type": "json_object"}, # Nếu model hỗ trợ và bạn muốn JSON object chứa array
        temperature=0. # Điều chỉnh để có sự đa dạng
    )
generated_json_string = response.choices[0].message.content

In [ ]:
# Save the generated JSON string to a file
with open("conversation_tone.json", "w") as f:
    f.write(generated_json_string)


In [12]:
cleaned_json_string = generated_json_string

# Remove the Markdown code block delimiters
# Common patterns are "```json\n" at the start and "\n```" at the end
# Or just "```json" and "```" if there are no newlines separating them from the content

# More robust cleaning:
# 1. Remove "```json" from the start.
# 2. Remove "```" from the end.
# 3. Strip any surrounding whitespace (including newlines) that might be left.

if cleaned_json_string.startswith("```json"):
    cleaned_json_string = cleaned_json_string[len("```json"):]
elif cleaned_json_string.startswith("```"): # If it's just ``` without 'json'
    cleaned_json_string = cleaned_json_string[len("```"):]

if cleaned_json_string.endswith("```"):
    cleaned_json_string = cleaned_json_string[:-len("```")]

# Strip whitespace (like newlines or spaces) from the beginning and end
cleaned_json_string = cleaned_json_string.strip()
# --- End of fix ---

try:
    parsed_data = json.loads(cleaned_json_string) # Use the cleaned string
    deepeval_test_cases = []
    for conv_data in parsed_data:
        llm_turns = []
        for turn_data in conv_data.get("turns", []):
            user_input = turn_data.get("user_input")
            gender_context_val = turn_data.get("user_gender_context")
            actual_output_placeholder = turn_data.get("bot_actual_output_placeholder", "")

            current_context = []
            if gender_context_val and gender_context_val != "null": #Check for string "null"
                current_context.append(f"User gender: {gender_context_val}")
            
            if user_input: #Only add turn if user_input exists
                llm_turns.append(
                    LLMTestCase(
                        input=user_input,
                        actual_output=actual_output_placeholder, # Will be filled later
                        context=current_context if current_context else None
                    )
                )
        if llm_turns: #Only add ConversationalTestCase if there are valid turns
            deepeval_test_cases.append(ConversationalTestCase(messages=llm_turns))

    print(f"Đã tạo thành công {len(deepeval_test_cases)} ConversationalTestCase objects.")
    # Now you can use deepeval_test_cases to run evaluations after filling actual_output

except json.JSONDecodeError as e:
    print(f"Lỗi giải mã JSON: {e}")
    # Print the cleaned string for better debugging if error persists
    print(f"Dữ liệu đã được làm sạch (một phần): {cleaned_json_string[:500]}...")
    print(f"Dữ liệu gốc (một phần): {generated_json_string[:500]}...")
except Exception as e:
    print(f"Lỗi không xác định khi xử lý: {e}")


Lỗi không xác định khi xử lý: ConversationalTestCase.__init__() got an unexpected keyword argument 'messages'


In [2]:
import json
from deepeval.test_case import LLMTestCase, ConversationalTestCase


try:
        #parsed_data = json.loads(generated_json_string)
        # load data from json file
        with open("conversation_tone.json", "r") as f:
            parsed_data = json.load(f)
        deepeval_test_cases = []
        for conv_data in parsed_data:
            llm_turns = []
            for turn_data in conv_data.get("turns", []):
                user_input = turn_data.get("user_input")
                gender_context_val = turn_data.get("user_gender_context")
                actual_output_placeholder = turn_data.get("bot_actual_output_placeholder", "")  #Nên là ""

                current_context = []
                if gender_context_val and gender_context_val != "null":  #Kiểm tra null dưới dạng chuỗi
                    current_context.append(f"User gender: {gender_context_val}")
            
                if user_input:  #Chỉ thêm turn nếu có user_input
                    llm_turns.append(
                        LLMTestCase(
                            input=user_input,
                            actual_output=actual_output_placeholder, # Sẽ được điền sau
                            context=current_context if current_context else None  # DeepEval có thể muốn None nếu context rỗng
                        )
                    )
            if llm_turns:  #Chỉ thêm ConversationalTestCase nếu có turns hợp lệ
                deepeval_test_cases.append(ConversationalTestCase(turns=llm_turns))  #DeepEval v0.20+ dùng messages
                                                                                    #(hoặc `turns=llm_turns` cho phiên bản cũ hơn)

        print(f"Đã tạo thành công {len(deepeval_test_cases)} ConversationalTestCase objects.")
        #Bây giờ bạn có thể dùng deepeval_test_cases để chạy đánh giá sau khi điền actual_output
except json.JSONDecodeError as e:
    print(f"Lỗi giải mã JSON: {e}")
    print(f"Dữ liệu nhận được: {generated_json_string[:500]}...")  #In một phần dữ liệu lỗi
except Exception as e:
    print(f"Lỗi không xác định khi xử lý: {e}")

Đã tạo thành công 30 ConversationalTestCase objects.


In [3]:
from deepeval.test_case import ConversationalTestCase, LLMTestCase
from deepeval.metrics import ConversationalGEval

from deepeval.test_case import LLMTestCaseParams
from deepeval import evaluate

In [4]:
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer
from uuid import uuid4
from models.user import UserRole    

def get_actual_answer(input: str, gender_context: str) -> str:
    user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
    )
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))

    # Add gender context to the conversation
    history = [
        {"role": "system", "content": "##BASE KNOWLEDGE:\n" + gender_context},
        {"role": "user", "content": str(input)}
    ]

    return gen_answer(
        thread_id=thread.id,
        history=history,
        user_id=user.id,
    )


2025-06-01 11:43:25 - Loaded .env file
2025-06-01 11:43:29 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-06-01 11:43:30 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
weave version 0.51.49 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
2025-06-01 11:43:31 - file_cache is only supported with oauth2client<4.0.0


In [5]:

pronoun_consistency_metric = ConversationalGEval(
    name="Vietnamese Pronoun Consistency",
    criteria="""Đánh giá khả năng của chatbot trong việc sử dụng đại từ nhân xưng tiếng Việt một cách chính xác và nhất quán trong suốt cuộc hội thoại. Cụ thể:
    1. Chatbot (assistant) trong 'actual_output' phải LUÔN LUÔN tự xưng là 'em'.  Ví dụ: 'dạ em chào anh', 'em có thể giúp gì ạ'. KHÔNG được dùng 'tôi', 'mình'
    2. Cách chatbot gọi người dùng (user) phải dựa trên thông tin được cung cấp hoặc cách người dùng tự xưng:
        - Nếu 'User gender' được cung cấp là 'male' hoặc người dùng tự xưng là 'anh', chatbot phải gọi người dùng là 'anh'.
        - Nếu 'User gender' được cung cấp là 'female' hoặc người dùng tự xưng là 'chị', chatbot phải gọi người dùng là 'chị'.
        - Nếu người dùng tự xưng là 'chú', chatbot phải gọi người dùng là 'chú' và tự xưng 'cháu'.
        - Nếu người dùng tự xưng là 'cô', chatbot phải gọi người dùng là 'cô' và tự xưng 'cháu'. 
        - Nếu 'User gender' là 'unknown' hoặc không được cung cấp, và người dùng không tự xưng theo một đại từ cụ thể nào ở trên, chatbot phải gọi người dùng là 'anh/chị'.
    3. Tính nhất quán: Chatbot phải duy trì cách xưng hô đã được thiết lập với người dùng một cách nhất quán trong các lượt trả lời tiếp theo trong cùng một cuộc hội thoại, trừ khi có thông tin mới rõ ràng thay đổi cách xưng hô.
    4. Không được sử dụng các cách xưng hô không phù hợp hoặc thiếu tôn trọng.
      HƯỚNG DẪN CHẤM ĐIỂM:
    - Điểm 1.0: Tuân thủ hoàn hảo tất cả các quy tắc trên trong mọi lượt của hội thoại.
    - Phạt nặng (điểm gần 0): Nếu chatbot tự xưng sai (ví dụ: xưng 'tôi' thay vì 'em').
    - Phạt nặng (điểm gần 0): Nếu chatbot gọi sai người dùng một cách rõ ràng (ví dụ: context là 'User gender: male' nhưng chatbot gọi là 'chị').
    - Phạt nhẹ hơn: Nếu chatbot dùng 'anh/chị' trong trường hợp có thể suy luận giới tính từ input nhưng không rõ ràng 100%.
    - Xem xét toàn bộ cuộc hội thoại để đánh giá tính nhất quán.
    """,
    
    evaluation_params=[
        LLMTestCaseParams.INPUT,            # Để phân tích input của user (ví dụ: "anh muốn hỏi...")
        LLMTestCaseParams.ACTUAL_OUTPUT,    # Để phân tích output của model
        LLMTestCaseParams.CONTEXT,        # Nếu thông tin `User gender` được truyền qua context cho mỗi lượt
    ]
    
)

In [6]:
tc = deepeval_test_cases[0]
for turn in tc.turns:
    print(turn.input)
    # turn list to str
    turn_str = ""
    for t in turn.context:
        turn_str += t
    print(turn_str)

Anh muốn hỏi cách tra cứu về hóa đơn đã mua hàng tại FPT Shop?
User gender: male
Nếu hóa đơn đó mua hơn 1 năm rồi thì có tìm lại được không shop?
User gender: male


In [7]:
def lst_context_to_str(lst: list) -> str:
    turn_str = ""
    for t in lst:
        turn_str += t
    return turn_str


In [8]:
tc = deepeval_test_cases[0]
for turn in tc.turns:
    turn.actual_output = get_actual_answer(turn.input, lst_context_to_str(turn.context))
    print(turn.actual_output)


🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01972b4e-23bb-7fe2-b9c9-7c4767e75b93
2025-06-01 11:43:53 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User request: {'user_demand': <ProductType.UNDETERMINED: 'undetermined'>, 'user_info': {'phone_number': None, 'email': None}}
Detect demand response: type='finished' content='The user request has been successfully processed.' instructions=[] UserIntent(is_user_needs_other_suggestions=False, product_type=None)
2025-06-01 11:43:54 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
faq id : 12
faq id : 16
faq id : 62
faq id : 18
2025-06-01 11:44:00 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Updating user memory with id: 594337d9-7381-4a39-a3b1-233965240458 and data: {'user_demand': None, 'product_name': None, 'brand_code': None, 'brand_name': None, 'min_price': None, 'max_price': None, 'phone_number': None, 'email': None, 'intent': {'is_user_need

In [9]:
pronoun_consistency_metric.measure(tc)
print(pronoun_consistency_metric.score, pronoun_consistency_metric.reason)

Output()

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01972b4e-93c8-7562-814c-7645e051be0a

2025-06-01 11:44:23 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01972b4e-a22c-7af2-b7ed-2c1477e1f6cf

2025-06-01 11:44:25 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


0.9999999999999998 The chatbot consistently uses 'em' for self-reference and 'anh' for the user, aligning with the user's male gender context. Pronoun usage is consistent and appropriate throughout the conversation.


In [32]:
evaluate(
    test_cases=tc,
    metrics=[pronoun_consistency_metric],
    
)

TypeError: 'ConversationalTestCase' object is not iterable

In [ ]:

# Define conversation starters
CONVERSATION_STARTERS = [
    "Chào shop, mình muốn hỏi về {topic}",
    "Shop ơi, cho mình hỏi về {topic}",
    "Mình đang tìm hiểu về {topic}",
    "Bên shop có {topic} không?",
    "Mình muốn hỏi về {topic}"
]

# Define follow-up questions
FOLLOW_UP_QUESTIONS = [
    "Vậy {topic} có {detail} không?",
    "Có thể cho mình biết thêm về {detail} không?",
    "Vậy {topic} thì {detail} thế nào?",
    "Có thể giải thích rõ hơn về {detail} không?",
    "Vậy {topic} có {detail} không shop?"
]
def generate_bot_response(user_message: str, faqs: List[FAQModel], turn_number: int) -> str:
    """Generate a bot response based on user message and FAQs"""
    if turn_number == 1:
        # First turn - greeting and asking for more information
        return f"Dạ chào anh/chị, em có thể giúp gì cho anh/chị ạ?"
    else:
        # Follow-up turns - provide information based on FAQs
        # Use OpenAI to generate a response based on the user message and FAQs
        faq_context = "\n".join([
            f"Q: {faq.question}\nA: {faq.answer}"
            for faq in faqs
        ])
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": f"""You are a helpful customer service representative at FPT Shop. 
                Use the following FAQs to answer customer questions:
                {faq_context}
                
                Guidelines:
                - Be professional and friendly
                - Use appropriate Vietnamese pronouns (em-anh/chị)
                - Keep responses concise but informative
                - If the question isn't covered in FAQs, ask for more details"""},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7,
            max_tokens=150
        )
        
        return response.choices[0].message.content

In [ ]:
def generate_conversation_test_case(faqs: List[FAQModel], num_turns: int = 2) -> ConversationalTestCase:
    """Generate a multi-turn conversation test case"""
    # Select random FAQ
    faq = random.choice(faqs)
    
    # Generate conversation turns
    turns = []
    
    # First turn
    first_input = random.choice(CONVERSATION_STARTERS).format(topic=faq.title)
    first_output = generate_bot_response(first_input, [faq], 1)
    turns.append(LLMTestCase(
        input=first_input,
        actual_output=first_output,
        context=[f"FAQ Category: {faq.category}"]
    ))
    
    # Follow-up turns
    for i in range(1, num_turns):
        # Extract key details from FAQ answer for follow-up questions
        follow_up = random.choice(FOLLOW_UP_QUESTIONS).format(
            topic=faq.title,
            detail=random.choice(faq.answer.split(".")[0].split())
        )
        bot_response = generate_bot_response(follow_up, [faq], i + 1)
        turns.append(LLMTestCase(
            input=follow_up,
            actual_output=bot_response,
            context=[f"FAQ Category: {faq.category}"]
        ))
    
    return ConversationalTestCase(turns=turns)

def generate_test_cases(num_cases: int = 10, min_turns: int = 2, max_turns: int = 4) -> List[ConversationalTestCase]:
    """Generate multiple test cases using FAQs"""
    all_test_cases = []
    faqs = get_all_faqs()
    
    for _ in range(num_cases):
        num_turns = random.randint(min_turns, max_turns)
        test_case = generate_conversation_test_case(faqs, num_turns)
        all_test_cases.append(test_case)
    
    return all_test_cases

def save_test_cases(test_cases: List[ConversationalTestCase], output_file: str):
    """Save test cases to a JSON file"""
    serialized_cases = []
    for case in test_cases:
        serialized_turns = []
        for turn in case.turns:
            serialized_turns.append({
                "input": turn.input,
                "actual_output": turn.actual_output,
                "context": turn.context
            })
        serialized_cases.append({"turns": serialized_turns})
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(serialized_cases, f, ensure_ascii=False, indent=2)